In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K
import time

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\toolkit

In [2]:
print(K.image_data_format())

channels_last


In [3]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [4]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [5]:
def DataSet(iFold,noise_tag,dup_tag,rand_tag):
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    num_class = 2
    
    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, num_feature), dtype=float)
    Y_train = np.empty((nrow_train_dup, num_class), dtype=float)

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(num_feature):
                X_train[sam_count,i] = float(data_train[index,i])
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, num_feature), dtype=float)
    Y_test  = np.empty((nrow_test, num_class), dtype=float)

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(num_feature):
            X_test[index,i] = float(data_test[index,i])
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
        
    if (int)(noise_tag):
        noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
        output = X_train + noise
        X_train = np.concatenate((X_train, output), axis=0)
        Y_train = np.concatenate((Y_train, Y_train), axis=0)
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    #X_train_new = np.empty((nrow_train_dup, 224, 224, 3))
    #Y_train_new = Y_train
    
    #for index in range(nrow_train_dup):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
    #    for c in range(3):
    #        X_train_new[index,:,:,c] = tmp_new[:,:,0]
    
    #X_test_new = np.empty((nrow_test, 224, 224, 3))
    #Y_test_new = Y_test
    
    #for index in range(nrow_test):
    #    tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
    #    for c in range(3):
    #        X_test_new[index,:,:,c] = tmp_new[:,:,0]
            
    #del X_train, X_test
    #gc.collect()

    return X_train,Y_train,X_test,Y_test

In [6]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [7]:
def neural(channel,length,classes):
    input_shape = (channel,length)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (length,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    #model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    #model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    #model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    #model.add(Flatten())
    #model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(600,activation=LeakyRelu,input_dim=length,name="fc1"))
    #model.add(Dense(600,activation=LeakyRelu,name="fc2"))
    #model.add(Dense(768,activation=LeakyRelu,name="fc3"))
    model.add(Dense(192,activation=LeakyRelu,name="fc4"))
    model.add(Dense(classes,activation="softmax",name="fc5"))
    return model


In [8]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_label = model.predict_classes(sam_data, batch_size=32, verbose=1)
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [9]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]
all_time_train=[]
all_time_predict_train=[]
all_time_predict_test=[]

nRun = 10
side_len = 22
num_class = 2
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for fold in range(nRun):
                iFold = fold + 1
                X_train,Y_train,X_test,Y_test = DataSet(iFold,noise_tag,dup_tag,ran_tag)
                print('X_train shape : ',X_train.shape)
                print('Y_train shape : ',Y_train.shape)
                print('X_test shape : ',X_test.shape)
                print('Y_test shape : ',Y_test.shape)
                #print(X_train[0])
                #print(Y_train[0,1])
                print(noise_tag,dup_tag,ran_tag,iFold)
                
                num_feature = X_train.shape[1]

                # # model
                #model = ResNet50(
                #    weights=None,
                #    classes=2
                #)

                ##
                model = neural(1,num_feature,num_class)

                model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # # train
                start = time.clock()
                model.fit(X_train, Y_train, epochs=1, batch_size=16)
                all_time_train.append(time.clock()-start)

                # # evaluate
                #model.evaluate(X_test, Y_test, batch_size=32)

                # #
                #models.append(model)

                # # save
                #save_path =("./my_resnet_model.h5_F%d") % (iFold)
                #model.save(save_path)

                #
                X_train,Y_train,X_test,Y_test = DataSet(iFold,0,0,ran_tag)
                start = time.clock()
                train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
                all_time_predict_train.append(time.clock()-start)
                start = time.clock()
                test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
                all_time_predict_test.append(time.clock()-start)
                all_TP_train.append(train_N_TP)
                all_FP_train.append(train_N_FP)
                all_TN_train.append(train_N_TN)
                all_FN_train.append(train_N_FN)
                all_count_train.append(train_count)
                all_TP_test.append(test_N_TP)
                all_FP_test.append(test_N_FP)
                all_TN_test.append(test_N_TN)
                all_FN_test.append(test_N_FN)
                all_count_test.append(test_count)


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 1
Epoch 1/1
1410/1410 [==============================] - 0s 77us/step
[1316.   94.]
157/157 [==============================] - 0s 45us/step
[147.  10.]


D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 2
Epoch 1/1
1410/1410 [==============================] - 0s 71us/step
[1316.   94.]
157/157 [==============================] - 0s 38us/step
[147.  10.]
X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 3
Epoch 1/1
1410/1410 [==============================] - 0s 94us/step
[1316.   94.]
157/157 [==============================] - 0s 57us/step
[147.  10.]
X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 4
Epoch 1/1
1410/1410 [==============================] - 0s 107us/step
[1317.   93.]
157/157 [==============================] - 0s 51us/step
[146.  11.]
X_train shape :  (1410, 474)
Y_train shape :  (1410, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 0 0 5
Epoch 1/1
1410/1410 [==============================] - 0s 128us/step
[1317.   93.]
15

Epoch 1/1
1410/1410 [==============================] - 1s 433us/step
[1317.   93.]
157/157 [==============================] - 0s 70us/step
[146.  11.]
X_train shape :  (2619, 468)
Y_train shape :  (2619, 2)
X_test shape :  (157, 468)
Y_test shape :  (157, 2)
0 1 0 6
Epoch 1/1
1410/1410 [==============================] - 1s 447us/step
[1317.   93.]
157/157 [==============================] - 0s 64us/step
[146.  11.]
X_train shape :  (2619, 474)
Y_train shape :  (2619, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
0 1 0 7
Epoch 1/1
1410/1410 [==============================] - 1s 462us/step
[1317.   93.]
157/157 [==============================] - 0s 77us/step
[146.  11.]
X_train shape :  (2633, 474)
Y_train shape :  (2633, 2)
X_test shape :  (156, 474)
Y_test shape :  (156, 2)
0 1 0 8
Epoch 1/1
1411/1411 [==============================] - 1s 486us/step
[1317.   94.]
156/156 [==============================] - 0s 64us/step
[146.  10.]
X_train shape :  (2633, 474)
Y_train shape :  (2

1411/1411 [==============================] - 1s 830us/step
[1317.   94.]
156/156 [==============================] - 0s 90us/step
[146.  10.]
X_train shape :  (2822, 474)
Y_train shape :  (2822, 2)
X_test shape :  (156, 474)
Y_test shape :  (156, 2)
1 0 0 9
Epoch 1/1
1411/1411 [==============================] - 1s 848us/step
[1317.   94.]
156/156 [==============================] - 0s 83us/step
[146.  10.]
X_train shape :  (2822, 474)
Y_train shape :  (2822, 2)
X_test shape :  (156, 474)
Y_test shape :  (156, 2)
1 0 0 10
Epoch 1/1
1411/1411 [==============================] - 1s 861us/step
[1317.   94.]
156/156 [==============================] - 0s 77us/step
[146.  10.]
X_train shape :  (2820, 474)
Y_train shape :  (2820, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
1 0 1 1
Epoch 1/1
1410/1410 [==============================] - 1s 887us/step
[1316.   94.]
157/157 [==============================] - 0s 76us/step
[147.  10.]
X_train shape :  (2820, 474)
Y_train shape :  (2820, 2)
X

1410/1410 [==============================] - 2s 1ms/step
[1316.   94.]
157/157 [==============================] - 0s 89us/step
[147.  10.]
X_train shape :  (5264, 474)
Y_train shape :  (5264, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
1 1 1 2
Epoch 1/1
1410/1410 [==============================] - 2s 1ms/step
[1316.   94.]
157/157 [==============================] - 0s 89us/step
[147.  10.]
X_train shape :  (5264, 474)
Y_train shape :  (5264, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
1 1 1 3
Epoch 1/1
1410/1410 [==============================] - 2s 1ms/step
[1316.   94.]
157/157 [==============================] - 0s 96us/step
[147.  10.]
X_train shape :  (5238, 474)
Y_train shape :  (5238, 2)
X_test shape :  (157, 474)
Y_test shape :  (157, 2)
1 1 1 4
Epoch 1/1
1410/1410 [==============================] - 2s 1ms/step
[1317.   93.]
157/157 [==============================] - 0s 102us/step
[146.  11.]
X_train shape :  (5238, 474)
Y_train shape :  (5238, 2)
X_test sh

In [10]:
noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
output = X_train + noise
X_train_all = np.concatenate((X_train, output), axis=0)
print(noise)
print(noise.shape)
print(output.shape)
print(X_train.shape)
print(X_train_all.shape)
for i in range(len(X_train)):
    if (X_train[i]==X_train_all[i]).all()==False:
        print(i)
for i in range(len(X_train)):
    ind = len(X_train)+i
    if (output[i]==X_train_all[ind]).all()==False:
        print(ind)
        

[[ 0.01994212 -0.00696343 -0.00523997 ... -0.00452903 -0.01114098
   0.00902715]
 [-0.00423169 -0.00951541  0.00186985 ... -0.0006949  -0.00141815
   0.012068  ]
 [-0.01311736 -0.0035457  -0.02044626 ...  0.00456607 -0.01328517
   0.00665908]
 ...
 [-0.00615402  0.01022989  0.00312129 ... -0.0122602  -0.0044113
  -0.02492393]
 [ 0.00803528  0.00027438  0.00581989 ... -0.01495759 -0.0072794
  -0.00499793]
 [ 0.00326157 -0.00095485  0.00466259 ...  0.01744827 -0.00422934
  -0.00140094]]
(1411, 474)
(1411, 474)
(1411, 474)
(2822, 474)


In [11]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_TP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
            for iRun in range(nRun):
                print(all_TP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])

for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_count_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_count_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[93.  0.]
[   0. 1317.]
[ 0. 93.]
[1317.    0.]
[94.  0.]
[   0. 1317.]
[ 0. 94.]
[1317.    0.]
[94.  0.]
[   0. 1317.]
[ 0. 94.]
[1317.    0.]
[94.  0.]
[   0. 1317.]
[ 0. 94.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[147.   0.]
[10.  0.]
[  0. 147.]
[ 0. 10.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[11.  0.]
[  0. 146.]
[ 0. 11.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1

[146.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1316.   94.]
[147.  10.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   93.]
[146.  11.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]
[1317.   94.]
[146.  10.]


In [12]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


3.4546605
1.4641785
1.4729935000000012
1.4935507000000001
1.7667542999999988
1.553881699999998
1.6236533000000009
1.8639927999999983
1.8492595000000023
1.7763583999999994
1.866920400000005
2.0371644000000018
1.9573378999999989
2.049051199999994
2.0489056999999917
2.101872600000007
2.3966068000000007
2.1840660999999955
2.2197093999999993
2.3256766000000084
2.678696500000001
2.7914597999999984
3.203546799999998
2.9324048999999945
3.093793699999992
3.028409200000027
3.010646300000019
3.132931900000017
3.3035634999999957
3.583728199999996
3.3828552999999886
3.394242399999996
3.391145800000004
3.499973600000004
3.6127810000000125
3.6453108999999984
3.7133082
3.6910516000000086
4.173255400000016
3.8450287000000003
4.0239455999999905
4.0411776999999915
4.3898902999999905
4.225529999999992
4.453438500000004
4.2363684000000035
4.327163499999983
4.4445852999999715
5.03315630000003
4.65603729999998
4.526267200000007
4.733131100000037
4.64309320000001
4.842316500000038
4.69484719999997
4.861046799

In [13]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.12281560000000002
0.11473090000000141
0.15004980000000145
0.16772070000000028
0.20097200000000015
0.20680180000000092
0.22174079999999918
0.2562938000000017
0.27023909999999773
0.3049229000000011
0.3112374000000031
0.32030519999999996
0.35174759999999594
0.3873664000000048
0.4104267999999962
0.4056601999999998
0.43380960000000357
0.454115299999998
0.48618630000001417
0.5099244000000027
0.528100699999996
0.533116899999996
0.5706997000000058
0.5811480999999787
0.6261381999999855
0.6472712999999999
0.6674179999999978
0.7032636999999795
0.719947000000019
0.7501189000000181
0.7500031999999806
0.7855273000000125
0.8356567000000155
0.8405172999999877
0.863181400000002
0.8742403999999908
0.9179957999999999
0.9398015000000157
0.9612159000000133
1.0002516000000128
1.0027156999999534
1.0440728000000377
1.0684415999999715
1.0756734999999935
1.1231988000000115
1.135819099999992
1.1386441999999874
1.1861820999999964
1.213295799999969
1.2306912000000239
1.2673763999999892
1.278495399999997
1.293711

In [14]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.010068699999999708
0.009520699999999493
0.012309000000000125
0.01362930000000162
0.01168350000000018
0.011992799999998027
0.012128399999994599
0.01418790000000314
0.011334899999994263
0.011384900000003029
0.011686899999993727
0.013180499999997153
0.012507100000007654
0.01172159999998712
0.015181299999994735
0.011792900000003215
0.011337999999994963
0.015286900000006653
0.011773399999995604
0.01298280000000318
0.015743600000007518
0.012611700000007886
0.013649900000018533
0.014731400000016492
0.01295679999998356
0.013036399999975856
0.015046200000000454
0.01312630000001036
0.014610599999997476
0.014846400000010362
0.014950299999981098
0.012711099999989983
0.013148099999995111
0.012399299999998448
0.016270800000000918
0.013590699999980416
0.0161945999999773
0.012899099999998498
0.0126657999999793
0.012896399999988262
0.01307269999995242
0.014211400000021968
0.013354499999991276
0.013919399999963389
0.014268500000014228
0.017791900000020178
0.015670699999986937
0.0168297000000166
0.0163

In [15]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))

#save_path =("./tmp_TP_EP%d") % (1)
#fl=open(save_path, 'w')
#lists=[line+"\n" for line in all_TP_train]
#fl.writelines(lists)